In [9]:
import dice_ml
from dice_ml.utils import helpers # helper functions
from sklearn.datasets import load_iris
from sklearn.svm import SVC
import pandas as pd
import tensorflow as tf

In [5]:
# print(sklearn.__version__) # 0.24.1

iris = load_iris()
X_iris = pd.DataFrame(iris.data, columns=iris.feature_names)
y_iris = pd.Series(iris.target, name='target')

clf = SVC()
clf.fit(X_iris, y_iris)

iris_df = X_iris.join(y_iris)
print(iris_df)

     sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)  \
0                  5.1               3.5                1.4               0.2   
1                  4.9               3.0                1.4               0.2   
2                  4.7               3.2                1.3               0.2   
3                  4.6               3.1                1.5               0.2   
4                  5.0               3.6                1.4               0.2   
..                 ...               ...                ...               ...   
145                6.7               3.0                5.2               2.3   
146                6.3               2.5                5.0               1.9   
147                6.5               3.0                5.2               2.0   
148                6.2               3.4                5.4               2.3   
149                5.9               3.0                5.1               1.8   

     target  
0         0  

In [6]:
d = dice_ml.Data(dataframe=iris_df,
                 continuous_features=iris.feature_names,
                 outcome_name='target')

m = dice_ml.Model(model=clf, backend='sklearn')
exp = dice_ml.Dice(d,m)

AttributeError: module 'tensorflow' has no attribute 'get_default_session'

In [12]:
import dice_ml
from dice_ml.utils import helpers # helper functions

# Dataset for training an ML model
d = dice_ml.Data(dataframe=helpers.load_adult_income_dataset(),
                 continuous_features=['age', 'hours_per_week'],
                 outcome_name='income')


backend = 'TF'+tf.__version__[0] # TF2 in your case
ML_modelpath = helpers.get_adult_income_modelpath(backend=backend)
m = dice_ml.Model(model_path= ML_modelpath, backend=backend)



# Pre-trained ML model
# m = dice_ml.Model(model_path=dice_ml.utils.helpers.get_adult_income_modelpath())
# DiCE explanation instance
exp = dice_ml.Dice(d,m)

Metal device set to: Apple M1 Max


2022-07-03 22:38:28.431853: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-07-03 22:38:28.431984: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [13]:
query_instance = {'age':22,
    'workclass':'Private',
    'education':'HS-grad',
    'marital_status':'Single',
    'occupation':'Service',
    'race': 'White',
    'gender':'Female',
    'hours_per_week': 45}

In [14]:
# Generate counterfactual examples
dice_exp = exp.generate_counterfactuals(query_instance, total_CFs=4, desired_class="opposite")
# Visualize counterfactual explanation
dice_exp.visualize_as_dataframe()

/Users/joaorulff/miniforge3/envs/cfsim/lib/python3.9/site-packages/dice_ml/data_interfaces/public_data_interface.py:582: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp = self.ohe_base_df.append(query_instance, ignore_index=True, sort=False)


Diverse Counterfactuals found! total time taken: 00 min 45 sec
Query instance (original outcome : 0)


/Users/joaorulff/miniforge3/envs/cfsim/lib/python3.9/site-packages/dice_ml/data_interfaces/public_data_interface.py:582: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp = self.ohe_base_df.append(query_instance, ignore_index=True, sort=False)
/Users/joaorulff/miniforge3/envs/cfsim/lib/python3.9/site-packages/dice_ml/data_interfaces/public_data_interface.py:582: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp = self.ohe_base_df.append(query_instance, ignore_index=True, sort=False)
/Users/joaorulff/miniforge3/envs/cfsim/lib/python3.9/site-packages/dice_ml/data_interfaces/public_data_interface.py:582: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp = self.ohe_base_df.append(query_instance, ignore_index=True, sort=False

,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,22.0,Private,HS-grad,Single,Service,White,Female,45.0,0.019



Diverse Counterfactual set (new outcome: 1.0)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,70.0,Private,Masters,Single,White-Collar,White,Female,51.0,0
1,22.0,Self-Employed,Doctorate,Married,Service,White,Female,45.0,0
2,47.0,Private,HS-grad,Married,Service,White,Female,45.0,0
3,36.0,Private,Prof-school,Married,Service,White,Female,62.0,0
